# GABRIEL Library Stress Test

This notebook exercises various API calls and pipelines using real OpenAI responses. Ensure you have API access before running. All calls set `use_dummy=False`.


In [ ]:

import os, sys, shutil, asyncio, pandas as pd
sys.path.insert(0, os.path.abspath('../src'))
from gabriel.utils import openai_utils
from gabriel.tasks import (
    Ratings, RatingsConfig,
    BasicClassifier, BasicClassifierConfig,
    Deidentifier, DeidentifyConfig,
    Regional, RegionalConfig,
    CountyCounter,
    EloRater, EloConfig,
    RecursiveEloConfig, RecursiveEloRater,
)
from gabriel.utils import Teleprompter, PromptParaphraser, PromptParaphraserConfig

out_dir = 'stress_test_outputs'
if os.path.exists(out_dir):
    shutil.rmtree(out_dir)
os.makedirs(out_dir, exist_ok=True)


In [ ]:

try:
    from sklearn.datasets import fetch_20newsgroups
    ng = fetch_20newsgroups(subset='train', categories=['sci.space'], remove=('headers','footers','quotes'), download_if_missing=False)
    sample_texts = ng.data[:8]
except Exception:
    sample_texts = [
        'Space exploration has entered a new golden age with private companies.',
        'Astronomers recently discovered water on a distant exoplanet.',
        'Rockets are becoming reusable thanks to modern engineering.',
        'Satellite technology enables global internet coverage.',
        'Mars missions are planned for the next decade.',
        'The International Space Station hosts numerous experiments.',
        'Studying cosmic radiation helps us understand the universe.',
        'Observatories around the world capture images of distant galaxies.',
    ]


In [ ]:

ng_df = await openai_utils.get_all_responses(
    prompts=sample_texts,
    identifiers=[f'ng{i}' for i in range(1, len(sample_texts)+1)],
    use_dummy=False,
    save_path=os.path.join(out_dir,'ng.csv')
)
ng_df


## Basic `get_all_responses`

In [ ]:

prompts = [
    'Summarize the plot of Hamlet in two sentences.',
    'Explain the theory of general relativity in simple terms.',
    'Describe how photosynthesis works.',
    'Give three reasons why exercise is beneficial.',
    'Outline the history of the Internet.',
    'What causes thunderstorms to form?',
    'Provide a short biography of Ada Lovelace.',
    'How does blockchain technology maintain security?',
    'List important features of a democratic government.',
    'Why is biodiversity important for ecosystems?'
]
df = await openai_utils.get_all_responses(
    prompts=prompts,
    identifiers=[f'p{i}' for i in range(1, len(prompts)+1)],
    use_dummy=False,
    save_path=os.path.join(out_dir,'basic.csv')
)
df


## JSON mode

In [ ]:

json_prompts = [
    'Create a JSON object with keys "name", "age", and "city" for a fictional person named Alice who is 30 and lives in Denver.',
    'Provide JSON describing a product with fields "id", "description", and "price" for a laptop that costs 999.'
]
schema = {"type": "object"}
json_df = await openai_utils.get_all_responses(
    prompts=json_prompts,
    json_mode=True,
    expected_schema=schema,
    use_dummy=False,
    save_path=os.path.join(out_dir,'json.csv')
)
json_df


## Web search tool usage

In [ ]:

search_prompts = [
    'What is the tallest mountain in Africa and who led the first recorded ascent?'
]
web_df = await openai_utils.get_all_responses(
    prompts=search_prompts,
    identifiers=['search1'],
    use_web_search=True,
    use_dummy=False,
    save_path=os.path.join(out_dir,'web.csv')
)
web_df


## Resume from existing results

In [ ]:

resume_prompts = [
    'Define artificial intelligence.',
    'What are neural networks used for?',
    'Explain the Turing test.',
    'Describe the concept of machine learning.',
    'List applications of computer vision.'
]
# First run with only three prompts
_ = await openai_utils.get_all_responses(
    prompts=resume_prompts[:3],
    identifiers=[f'r{i}' for i in range(1,4)],
    use_dummy=False,
    save_path=os.path.join(out_dir,'resume.csv')
)
# Second run with all prompts (should only process remaining ones)
resume_df = await openai_utils.get_all_responses(
    prompts=resume_prompts,
    identifiers=[f'r{i}' for i in range(1,6)],
    use_dummy=False,
    save_path=os.path.join(out_dir,'resume.csv')
)
resume_df


## Ratings pipeline

In [ ]:

data = pd.DataFrame({'text': [
    'The new smartphone release offers cutting-edge features and sleek design.',
    'Customer support was unhelpful and slow to respond.'
]})
ratings_cfg = RatingsConfig(
    attributes={
        'clarity': 'Is the text easy to understand?',
        'detail': 'Does the text provide sufficient detail?',
        'tone': 'Assess the overall tone.'
    },
    save_dir=os.path.join(out_dir,'ratings'),
    use_dummy=False
)
ratings_res = await Ratings(ratings_cfg).run(data, text_column='text')
ratings_res


## BasicClassifier pipeline

In [ ]:

clf_data = pd.DataFrame({'txt': [
    'I absolutely loved the new movie!',
    'The service at the restaurant was terrible.',
    'It was an average experience overall.'
]})
clf_cfg = BasicClassifierConfig(
    labels={
        'positive': 'Is the sentiment positive?',
        'negative': 'Is the sentiment negative?',
        'neutral': 'Is the sentiment neutral?'
    },
    save_dir=os.path.join(out_dir,'classifier'),
    use_dummy=False
)
clf_res = await BasicClassifier(clf_cfg).run(clf_data, text_column='txt')
clf_res


## Deidentifier pipeline

In [ ]:

deid_data = pd.DataFrame({'text': [
    'Jane Doe met with Dr. Smith in New York on July 4th, 2021.',
    'Contact me at 555-123-4567 or email john@example.com.'
]})
deid_cfg = DeidentifyConfig(
    save_path=os.path.join(out_dir,'deid.csv'),
    use_dummy=False
)
deid_res = await Deidentifier(deid_cfg).run(deid_data, text_column='text')
deid_res


## Regional analysis pipeline

In [ ]:

reg_data = pd.DataFrame({'county': ['A', 'B', 'C']})
reg_cfg = RegionalConfig(
    save_dir=os.path.join(out_dir,'regional'),
    use_dummy=False
)
regional_task = Regional(reg_data, 'county', topics=['economy', 'public safety'], cfg=reg_cfg)
regional_res = await regional_task.run()
regional_res


## CountyCounter pipeline

In [ ]:

county_data = pd.DataFrame({
    'county': ['A', 'B', 'C'],
    'fips': ['00001', '00002', '00003']
})
cc = CountyCounter(
    county_data,
    county_col='county',
    topics=['econ', 'health'],
    fips_col='fips',
    save_dir=os.path.join(out_dir,'county'),
    use_dummy=False,
    n_elo_rounds=1
)
county_res = await cc.run()
county_res


## EloRater pipeline

In [ ]:

elo_data = pd.DataFrame({'identifier':['x','y'], 'text':['Text X with clear explanation','Text Y with vague content']})
tele = Teleprompter()
elo_cfg = EloConfig(
    attributes={'clarity':'', 'informativeness':''},
    n_rounds=2,
    save_dir=os.path.join(out_dir,'elo'),
    use_dummy=False
)
elo_task = EloRater(tele, elo_cfg)
elo_res = await elo_task.run(elo_data, text_col='text', id_col='identifier')
elo_res


## RecursiveEloRater pipeline

In [ ]:

rec_data = pd.DataFrame({'identifier':['a','b','c','d'], 'text':['Alpha text','Bravo text','Charlie text','Delta text']})
base_cfg = EloConfig(
    attributes={'score':''},
    n_rounds=1,
    save_dir=os.path.join(out_dir,'rec'),
    use_dummy=False
)
rec_cfg = RecursiveEloConfig(base_cfg=base_cfg, min_remaining=2)
rec_task = RecursiveEloRater(tele, rec_cfg)
rec_res = await rec_task.run(rec_data, text_col='text', id_col='identifier')
rec_res


In [ ]:

from gabriel.utils.parsing import safest_json
from unittest.mock import patch

async def fake_response(*args, **kwargs):
    return ['{"good": true}'], 0.0

with patch('gabriel.utils.openai_utils.get_response', fake_response):
    fixed_json = await safest_json('{bad:1}')
fixed_json


## PromptParaphraser

In [ ]:

batch_prompts = [f'Batch prompt {i}' for i in range(1,9)]
batch_df = await openai_utils.get_all_responses(
    prompts=batch_prompts,
    identifiers=[f'b{i}' for i in range(1,9)],
    use_batch=True,
    use_dummy=False,
    save_path=os.path.join(out_dir,'batch.csv'),
)
batch_df


In [ ]:

parap_cfg = PromptParaphraserConfig(
    n_variants=3,
    save_dir=os.path.join(out_dir,'parap'),
    use_dummy=False
)
parap = PromptParaphraser(parap_cfg)
parap_res = await parap.run(Ratings, ratings_cfg, data, text_column='text')
parap_res
